In [15]:
from Antropic.AnthropicsApiRequest import AnthropicsApiRequest
from Antropic.AnthropicsApiService import AnthropicsApiService
from CustomException import CustomException
from Payload import Message, Payload

import mimetypes
import datetime
import requests
import datetime
import base64
import json
import io
from PIL import Image
import numpy as np

In [81]:
from dotenv import dotenv_values

In [2]:
#import magic
from pdf2image import convert_from_bytes

In [82]:
config = dotenv_values(".env")

In [83]:
API_KEY_ANTROPIC = config.get("ANTHROPIC_API_KEY")

In [4]:
api_service = AnthropicsApiService(API_KEY_ANTROPIC) 

In [85]:
IMAGE_PATH = "./data/Elena_report_31032024.pdf"

In [86]:
with open(IMAGE_PATH, "rb") as f:
    im_bytes = f.read()        
im_b64 = base64.b64encode(im_bytes)

In [87]:
im_b64[:100]

b'JVBERi0xLjcNMSAwIG9iag08PC9UeXBlIC9YT2JqZWN0IC9TdWJ0eXBlIC9JbWFnZSAvTmFtZSAvSW0xIC9XaWR0aCAyNTQ0IC9I'

In [88]:
extraction_contract= ""

In [89]:
# Prepare the prompt
prompt = f"###return the content\n{extraction_contract}"

# Build the messages
messages = [Message(role="user", content=prompt)]

In [90]:
# Set the model
model = "claude-3-haiku-20240307"

In [91]:
# Create an instance of AnthropicsApiRequest
api_request = AnthropicsApiRequest(
        model=model,
        max_tokens=4096,
        messages=messages,
        system="You are a server API that receives an image and returns a JSON object with the the whole content of the file supplied. No translation"
    )

In [92]:
pdf_file = convert_from_bytes(im_bytes, dpi=300, grayscale=True)

In [93]:
pages= []
for i, page in enumerate(pdf_file):
    page_arr = np.asarray(page)
    im = Image.fromarray(page_arr)
    pages.append(im)
    im.save(f"./data/ER_{i}.jpg")

In [94]:
len(pages)

2

In [113]:
type(pages[0])

PIL.Image.Image

In [114]:
file_path = "./data/ER_0.jpg"

In [115]:
# Convert the file to base64
with open(file_path, "rb") as f:
    base64_encoded_image = base64.b64encode(f.read()).decode()

In [116]:
# Send the data to the Anthropics service and get the response
response = api_service.send_image_message(api_request, base64_encoded_image)

Response time: 10.777908563613892 seconds


In [117]:
response

'{\n  "learner_name": "Olena Nazarenko",\n  "student_number": "2023OSC07036",\n  "work_place_assessor_name": "N Russell",\n  "date": "7.03",\n  "method_of_assessment": [\n    "Direct observation",\n    "Reflection on own practice",\n    "Professional discussion"\n  ],\n  "recording_evidence": "Last week I worked with a SEN child 1:1. One day he refused to do his math work on the iPad and tried to download games instead. He did not want to give the iPad back and tried to hit me. Finally, I took the iPad away and explained to him that he could get that iPad for playing only at playtime when he finished his work.\\nWhen this boy heard this, he got angry and began to throw pencils, books, and paper everywhere, screaming loudly. I did not give him any reaction and said quietly \\"no\\". Then I brought him to the SEN room. There I could manage to switch his attention and gave him a new fidget toy. Next week he started Monday with math work on the iPad and did not try to play again.\\nMeanwhi

In [118]:
file_path = "./data/ER_1.jpg"

In [119]:
# Convert the file to base64
with open(file_path, "rb") as f:
    base64_encoded_image = base64.b64encode(f.read()).decode()

In [120]:
# Send the data to the Anthropics service and get the response
response = api_service.send_image_message(api_request, base64_encoded_image)

Response time: 4.361684560775757 seconds


In [121]:
response

'{\n  "image_content": "The image shows a handwritten feedback form. The text states that the person followed the implemented procedures for challenging behavior well, and they remained calm and applied the agreed strategies such as distraction techniques and application of reward system effectively to de-escalate challenging situations. The form is signed by the learner on 14.03 and the assessor on 15.03."\n}'